In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("assignment").getOrCreate()

In [ ]:
import urllib.request

def download_from_gdrive(file_id: str, output_path: str):
    """Download a public Google Drive file using urllib."""
    url = f"https://drive.google.com/uc?export=download&id={file_id}"
    try:
        urllib.request.urlretrieve(url, output_path)
        print(f"✅ Downloaded to: {output_path}")
    except Exception as e:
        print(f"❌ Failed to download {file_id}: {e}")

def load_csv_to_spark(path: str):
    """Load CSV file into Spark DataFrame."""
    try:
        df = spark.read.csv(path, header=True, inferSchema=True)
        print(f"✅ Loaded CSV into Spark: {path}")
        return df
    except Exception as e:
        print(f"❌ Failed to load CSV {path}: {e}")
        return None

# === Define files ===
files_to_download = {
    "transactions": {
        "file_id": "1AGXVlDhbMbhoGXDJG0IThnqz86Qy3hqb",
        "path": "/tmp/transactions.csv"
    },
    "cust_imp": {
        "file_id": "1abe9EkM_uf2F2hjEkbhMBG9Mf2dFE4Wo",
        "path": "/tmp/CustomerImportance.csv"
    }
}

# === Download and Load ===
dataframes = {}
for name, info in files_to_download.items():
    download_from_gdrive(info["file_id"], info["path"])
    dataframes[name] = load_csv_to_spark(info["path"])

# Access your dataframes:
transactions = dataframes["transactions"]
cust_imp = dataframes["cust_imp"]

# Quick preview
transactions.show(3)
cust_imp.show(3)


✅ Downloaded to: /tmp/transactions.csv
✅ Loaded CSV into Spark: /tmp/transactions.csv
✅ Downloaded to: /tmp/CustomerImportance.csv
✅ Loaded CSV into Spark: /tmp/CustomerImportance.csv
+----+-------------+---+------+----------+-------------+-----------+-------------------+------+-----+
|step|     customer|age|gender|zipcodeOri|     merchant|zipMerchant|           category|amount|fraud|
+----+-------------+---+------+----------+-------------+-----------+-------------------+------+-----+
|   0|'C1093826151'|'4'|   'M'|   '28007'| 'M348934600'|    '28007'|'es_transportation'|  4.55|    0|
|   0| 'C352968107'|'2'|   'M'|   '28007'| 'M348934600'|    '28007'|'es_transportation'| 39.68|    0|
|   0|'C2054744914'|'4'|   'F'|   '28007'|'M1823072687'|    '28007'|'es_transportation'| 26.89|    0|
+----+-------------+---+------+----------+-------------+-----------+-------------------+------+-----+
only showing top 3 rows

+-------------+-------------+------+-------------------+-----+
|       Source

In [ ]:
#transactions = spark.read.csv("/content/transactions.csv", header=True, inferSchema=True)
#cust_imp = spark.read.csv("/content/CustomerImportance.csv", header=True, inferSchema=True)

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window

ist_time = F.from_utc_timestamp(F.current_timestamp(), "Asia/Kolkata")


In [ ]:
transactions_clean = transactions.select(
    F.col("customer").alias("customerId"),
    F.col("merchant").alias("merchantId"),
    F.col("category").alias("transactionType")
)

cust_imp_clean = cust_imp.select(
    F.col("Source").alias("customerId"),
    F.col("Target").alias("merchantId"),
    F.col("Weight").cast("double").alias("weight"),
    F.col("typeTrans").alias("transactionType")
)

In [ ]:
# Step 1:
# Total transactions per merchant
merchant_txn_counts = transactions_clean.groupBy("merchantId").agg(
    F.count("*").alias("total_txns")
)

# filter merchants with ≥ 50k transactions
eligible_merchants = merchant_txn_counts.filter("total_txns >= 50000")

# Join to transactions to keep only eligible merchants
filtered_txns = transactions_clean.join(
    eligible_merchants.select("merchantId"), on="merchantId", how="inner"
)


In [ ]:
merchant_txn_counts.show(2)

+------------+----------+
|  merchantId|total_txns|
+------------+----------+
|'M857378720'|       122|
| 'M97925176'|       599|
+------------+----------+
only showing top 2 rows



In [ ]:
eligible_merchants.show(2)

+-------------+----------+
|   merchantId|total_txns|
+-------------+----------+
| 'M348934600'|    205426|
|'M1823072687'|    299693|
+-------------+----------+



In [ ]:
filtered_txns.show(2)

+------------+-------------+-------------------+
|  merchantId|   customerId|    transactionType|
+------------+-------------+-------------------+
|'M348934600'|'C1093826151'|'es_transportation'|
|'M348934600'| 'C352968107'|'es_transportation'|
+------------+-------------+-------------------+
only showing top 2 rows



In [ ]:
# Step 2:
# Count of transactions per customer per merchant
cust_txn_counts = filtered_txns.groupBy("merchantId", "customerId").agg(
    F.count("*").alias("txn_count")
)

# Window for percentile calculation per merchant
txn_window = Window.partitionBy("merchantId").orderBy(F.desc("txn_count"))

# Add raw percentile rank
cust_txn_counts = cust_txn_counts.withColumn(
    "txn_percentile_rank_raw",
    F.percent_rank().over(txn_window)
)

# Round to 4 decimal places to remove scientific notation
cust_txn_counts = cust_txn_counts.withColumn(
    "txn_percentile_rank", F.round("txn_percentile_rank_raw", 4)
).drop("txn_percentile_rank_raw")

# Top 10% customers by txn count
top_txn_customers = cust_txn_counts.filter(F.col("txn_percentile_rank") <= 0.1)

In [ ]:
cust_txn_counts.show(2)

+-------------+------------+---------+-------------------+
|   merchantId|  customerId|txn_count|txn_percentile_rank|
+-------------+------------+---------+-------------------+
|'M1823072687'|'C222098023'|      163|                0.0|
|'M1823072687'|'C185820405'|      158|             3.0E-4|
+-------------+------------+---------+-------------------+
only showing top 2 rows



In [ ]:
top_txn_customers.show(3)

+-------------+------------+---------+-------------------+
|   merchantId|  customerId|txn_count|txn_percentile_rank|
+-------------+------------+---------+-------------------+
|'M1823072687'|'C222098023'|      163|                0.0|
|'M1823072687'|'C185820405'|      158|             3.0E-4|
|'M1823072687'|'C333484923'|      156|             6.0E-4|
+-------------+------------+---------+-------------------+
only showing top 3 rows



In [ ]:
# Step 3:
# Join with cust_imp to get average weight
joined = top_txn_customers.join(
    cust_imp_clean,
    on=["merchantId", "customerId"],
    how="inner"
)

avg_weight_df = joined.groupBy("merchantId", "customerId").agg(
    F.avg("weight").alias("avg_weight")
)

# Window to rank avg_weight per merchant
weight_window = Window.partitionBy("merchantId").orderBy("avg_weight")

avg_weight_df = avg_weight_df.withColumn(
    "weight_percentile_rank",
    F.percent_rank().over(weight_window)
)

# Bottom 10% by weight
final_pat1 = avg_weight_df.filter("weight_percentile_rank <= 0.1")


In [ ]:
joined.show(2)

+-------------+-------------+---------+-------------------+------+-------------------+
|   merchantId|   customerId|txn_count|txn_percentile_rank|weight|    transactionType|
+-------------+-------------+---------+-------------------+------+-------------------+
| 'M348934600'|'C1861439904'|      113|             0.0843| 16.18|'es_transportation'|
|'M1823072687'|'C1934785516'|      146|             0.0168| 22.57|'es_transportation'|
+-------------+-------------+---------+-------------------+------+-------------------+
only showing top 2 rows



In [ ]:
avg_weight_df.show(2)

+-------------+-------------+------------------+----------------------+
|   merchantId|   customerId|        avg_weight|weight_percentile_rank|
+-------------+-------------+------------------+----------------------+
|'M1823072687'|'C1214229415'| 22.57316176470588|                   0.0|
|'M1823072687'|'C2035771335'|23.804571428571435|  0.002597402597402...|
+-------------+-------------+------------------+----------------------+
only showing top 2 rows



In [ ]:
final_pat1.show(2)

+-------------+-------------+------------------+----------------------+
|   merchantId|   customerId|        avg_weight|weight_percentile_rank|
+-------------+-------------+------------------+----------------------+
|'M1823072687'|'C1214229415'| 22.57316176470588|                   0.0|
|'M1823072687'|'C2035771335'|23.804571428571435|  0.002597402597402...|
+-------------+-------------+------------------+----------------------+
only showing top 2 rows



In [ ]:
result_pat1 = final_pat1.withColumn("patternId", F.lit("PatId1")) \
    .withColumn("actionType", F.lit("UPGRADE")) \
    .withColumn("YStartTime", ist_time) \
    .withColumn("detectionTime", ist_time) \
    .withColumn("customerName", F.col("customerId")) \
    .select(
        "YStartTime", "detectionTime", "patternId", "actionType",
        "customerName", "merchantId"
    )

  # result_pat1 holds the customer name that the merchant wants to upgrade
  #

In [ ]:
result_pat1.show(5)

+--------------------+--------------------+---------+----------+-------------+-------------+
|          YStartTime|       detectionTime|patternId|actionType| customerName|   merchantId|
+--------------------+--------------------+---------+----------+-------------+-------------+
|2025-07-11 19:11:...|2025-07-11 19:11:...|   PatId1|   UPGRADE|'C1214229415'|'M1823072687'|
|2025-07-11 19:11:...|2025-07-11 19:11:...|   PatId1|   UPGRADE|'C2035771335'|'M1823072687'|
|2025-07-11 19:11:...|2025-07-11 19:11:...|   PatId1|   UPGRADE|  'C15276068'|'M1823072687'|
|2025-07-11 19:11:...|2025-07-11 19:11:...|   PatId1|   UPGRADE| 'C861002792'|'M1823072687'|
|2025-07-11 19:11:...|2025-07-11 19:11:...|   PatId1|   UPGRADE| 'C566598245'|'M1823072687'|
+--------------------+--------------------+---------+----------+-------------+-------------+
only showing top 5 rows



PatId2

A customer whose average transaction value for a given merchant < Rs 23 and made at least 80 transactions with that merchant, merchant wants to mark them as CHILD(actionType) asap.

In [ ]:
transactions_clean = transactions.select(
    F.col("customer").alias("customerId"),
    F.col("merchant").alias("merchantId"),
    F.col("amount").cast("double").alias("amount")
)

# group by customers and merchants
# we get custid, merchantid, transactions count, avg amount
cust_merchant_stats = transactions_clean.groupBy("customerId", "merchantId").agg(
    F.count("*").alias("txn_count"),
    F.avg("amount").alias("avg_amount")
)

# avg transaction < 23 and atleast 80 transactions
pat2_result = cust_merchant_stats.filter(
    (F.col("txn_count") >= 80) &
    (F.col("avg_amount") < 23)
)

In [ ]:
cust_merchant_stats.show(2)

+------------+------------+---------+-----------------+
|  customerId|  merchantId|txn_count|       avg_amount|
+------------+------------+---------+-----------------+
|'C897057254'|'M348934600'|       41|27.27463414634146|
|'C394979408'|'M348934600'|       93|28.32763440860216|
+------------+------------+---------+-----------------+
only showing top 2 rows



In [ ]:
pat2_result.show(2)

+-------------+-------------+---------+------------------+
|   customerId|   merchantId|txn_count|        avg_amount|
+-------------+-------------+---------+------------------+
|  'C71938921'| 'M348934600'|      112|22.461785714285703|
|'C1799527037'|'M1823072687'|      103| 22.75106796116505|
+-------------+-------------+---------+------------------+
only showing top 2 rows



In [ ]:
# add required fields like time, pattern id and action type
result_pat2 = pat2_result.withColumn("patternId", F.lit("PatId2")) \
    .withColumn("actionType", F.lit("CHILD")) \
    .withColumn("YStartTime", ist_time) \
    .withColumn("detectionTime", ist_time) \
    .withColumn("customerName", F.col("customerId")) \
    .select(
        "YStartTime", "detectionTime", "patternId", "actionType",
        "customerName", "merchantId"
    )

In [ ]:
result_pat2.show(5)

+--------------------+--------------------+---------+----------+-------------+-------------+
|          YStartTime|       detectionTime|patternId|actionType| customerName|   merchantId|
+--------------------+--------------------+---------+----------+-------------+-------------+
|2025-07-11 19:12:...|2025-07-11 19:12:...|   PatId2|     CHILD|  'C71938921'| 'M348934600'|
|2025-07-11 19:12:...|2025-07-11 19:12:...|   PatId2|     CHILD|'C1799527037'|'M1823072687'|
|2025-07-11 19:12:...|2025-07-11 19:12:...|   PatId2|     CHILD| 'C747353905'|'M1823072687'|
|2025-07-11 19:12:...|2025-07-11 19:12:...|   PatId2|     CHILD| 'C474891377'|'M1823072687'|
|2025-07-11 19:12:...|2025-07-11 19:12:...|   PatId2|     CHILD|'C1098443227'|'M1823072687'|
+--------------------+--------------------+---------+----------+-------------+-------------+
only showing top 5 rows



PAT ID3

In [ ]:
gender_df = transactions.select(
    F.expr("substring(merchant, 2, length(merchant) - 2)").alias("merchantId"),
    F.expr("substring(customer, 2, length(customer) - 2)").alias("customerId"),
    F.expr("substring(gender, 2, 1)").alias("gender")  # 'F' → F, 'M' → M
)

# Count distinct genders per customer per merchant
gender_clean_check = gender_df.groupBy("merchantId", "customerId") \
    .agg(F.countDistinct("gender").alias("gender_type_count"))

# Keep only clean records (1 gender only per customer per merchant)
clean_customers = gender_clean_check.filter("gender_type_count = 1") \
    .select("merchantId", "customerId")

# Join back to filter ambiguous entries
gender_cleaned = gender_df.join(clean_customers, on=["merchantId", "customerId"], how="inner")


In [ ]:
# Remove duplicate customer-merchant-gender combos
unique_pairs = gender_cleaned.select("merchantId", "customerId", "gender").distinct()

# Pivot to get gender counts
gender_counts = unique_pairs.groupBy("merchantId") \
    .pivot("gender", ["F", "M"]).count().fillna(0)


In [ ]:
gender_counts.show(5)

+-----------+---+---+
| merchantId|  F|  M|
+-----------+---+---+
| M692898500|327|220|
| M732195782|223|179|
| M117188757|  8| 12|
|M1535107174|598|433|
| M547558035|366|250|
+-----------+---+---+
only showing top 5 rows



In [ ]:
dei_merchants = gender_counts.filter(
    (F.col("F") > 100) & (F.col("F") < F.col("M"))
)


In [ ]:
dei_merchants.show()

+----------+---+---+
|merchantId|  F|  M|
+----------+---+---+
|M677738360|173|174|
+----------+---+---+



In [ ]:
result_pat3 = dei_merchants.withColumn("patternId", F.lit("PatId3")) \
    .withColumn("actionType", F.lit("DEI-NEEDED")) \
    .withColumn("YStartTime", ist_time) \
    .withColumn("detectionTime", ist_time) \
    .withColumn("customerName", F.lit("")) \
    .select(
        "YStartTime", "detectionTime", "patternId", "actionType",
        "customerName", "merchantId"
    )

In [ ]:
result_pat3.show()
print("Pattern 3 count:", result_pat3.count())

# Verify gender distribution for one detected merchant
sample = result_pat3.select("merchantId").limit(1).collect()[0]["merchantId"]
unique_pairs.filter(F.col("merchantId") == sample).groupBy("gender").count().show()



+--------------------+--------------------+---------+----------+------------+----------+
|          YStartTime|       detectionTime|patternId|actionType|customerName|merchantId|
+--------------------+--------------------+---------+----------+------------+----------+
|2025-07-11 19:12:...|2025-07-11 19:12:...|   PatId3|DEI-NEEDED|            |M677738360|
+--------------------+--------------------+---------+----------+------------+----------+

Pattern 3 count: 1
+------+-----+
|gender|count|
+------+-----+
|     F|  173|
|     E|    1|
|     M|  174|
+------+-----+



In [ ]:
from functools import reduce

combined_df = reduce(
    lambda df1, df2: df1.unionByName(df2),
    [result_pat1, result_pat2, result_pat3]
)


In [ ]:
combined_df.show(10, truncate=False)
print("Total combined detections:", combined_df.count())
combined_df.groupBy("patternId").count().show()


+--------------------------+--------------------------+---------+----------+-------------+-------------+
|YStartTime                |detectionTime             |patternId|actionType|customerName |merchantId   |
+--------------------------+--------------------------+---------+----------+-------------+-------------+
|2025-07-11 19:12:52.522472|2025-07-11 19:12:52.522472|PatId1   |UPGRADE   |'C1214229415'|'M1823072687'|
|2025-07-11 19:12:52.522472|2025-07-11 19:12:52.522472|PatId1   |UPGRADE   |'C2035771335'|'M1823072687'|
|2025-07-11 19:12:52.522472|2025-07-11 19:12:52.522472|PatId1   |UPGRADE   |'C15276068'  |'M1823072687'|
|2025-07-11 19:12:52.522472|2025-07-11 19:12:52.522472|PatId1   |UPGRADE   |'C861002792' |'M1823072687'|
|2025-07-11 19:12:52.522472|2025-07-11 19:12:52.522472|PatId1   |UPGRADE   |'C566598245' |'M1823072687'|
|2025-07-11 19:12:52.522472|2025-07-11 19:12:52.522472|PatId1   |UPGRADE   |'C1433843082'|'M1823072687'|
|2025-07-11 19:12:52.522472|2025-07-11 19:12:52.522472|